In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Jan 23

@author: yaning
"""

import importlib
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math

# my own class files
import with_learning.learning_NN.Receptors as Receptors
import with_learning.learning_NN.Network as Network


importlib.reload(Receptors)
importlib.reload(Network)

path = "/home/yaning/Documents/Spiking_NN/with_learning/"
np.set_printoptions(threshold=np.inf)



In [ ]:
with open(path + "XOR_dataset.pkl", "rb") as f:
    dataset, binary = pickle.load(f)

In [ ]:
pointCount = 6000
deltaTms = 0.05
times = np.arange(pointCount) * deltaTms
initial_Vm = 1.3458754117369027

In [ ]:
# instead of using control i write those two functions :(have to copy it everytime i run)
all_synapses = []


def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def create_synapse(send_neuron, receive_neuron, type):
    
    # create receptors accordingly
    if type == "AMPA":
        # temporal solution for weight randomise
        # Receptors.LigandGatedChannelFactory.set_params()
        ampa_receptor = Receptors.AMPA(0.072, 1, -70, 1.35, 0.9, 1, 1, 1, 12, 10, 20, 10, 35, 7, 0.7, "AMPA")
        synapse = Network.Synapse(0.05, 0, send_neuron, receive_neuron, ampa_receptor)
        
    elif type == "AMPA+NMDA":
        # Receptors.LigandGatedChannelFactory.set_params()
        ampa_receptor = Receptors.AMPA(0.072, 1, -70, 1.35, 0.9, 1, 1, 1, 12, 10, 20, 10, 35, 7, 0.7, "AMPA")
        nmda_receptor = Receptors.NMDA(0.0012, 1, -70, 1.35, 0.9, 1, 1, 1, 13, 10, 20, 10, 15, 7, 0.7, "NMDA")
        synapse = Network.Synapse(0.05, 0, send_neuron, receive_neuron, ampa_receptor, nmda_receptor)
    
    elif type == "GABA":
        # Receptors.LigandGatedChannelFactory.set_params()
        # print(Receptors.LigandGatedChannelFactory.w_init_GABA)
        gaba_receptor = Receptors.GABA(0.004, 1, -140, 1.35, 0.9, 1, 1, 1, 12, 10, 20, 10, 20, 7, 0.7, "GABA")
        synapse = Network.Synapse(0.05, 0, send_neuron, receive_neuron, gaba_receptor)

    send_neuron.outgoing_synapses.append(synapse)
    receive_neuron.incoming_synapses.append(synapse)

    all_synapses.append(synapse)

def update_synapse_initial_values(infer_params):
    for synapse in all_synapses:
        for receptor in synapse.receptors:
            receptor.Vm = initial_Vm
            receptor.gP = 1
            
            receptor.e = infer_params["e"]
            receptor.u_se = infer_params["u_se"]
            receptor.g_decay = infer_params["g_decay"]
            receptor.g_rise = infer_params["g_rise"]
            receptor.w = infer_params["w"]
            receptor.tau_rec = infer_params["tau_rec"]
            receptor.tau_pre = infer_params["tau_pre"]
            receptor.tau_post = infer_params["tau_post"]

            if receptor.label == "GABA":
                receptor.gMax = infer_params["gMax_GABA"]
                receptor.tau_decay = infer_params["tau_decay_GABA"]
                receptor.tau_rise = infer_params["tau_rise_GABA"]
            
            elif receptor.label == "NMDA":
                receptor.tau_decay = infer_params["tau_decay_NMDA"]
                receptor.tau_rise = infer_params["tau_rise_NMDA"]
            
            elif receptor.label == "AMPA":
                receptor.tau_decay = infer_params["tau_decay_AMPA"]
                receptor.tau_rise = infer_params["tau_rise_AMPA"]

In [ ]:
# with learning every group has more neurons 
neuron_input_0 = Network.Neuron(deltaTms, 0, initial_Vm, "input_0")
neuron_input_1 = Network.Neuron(deltaTms, 0, initial_Vm, "input_1")
neuron_input_2 = Network.Neuron(deltaTms, 0, initial_Vm, "input_2")
# neuron_input_3 = Network.Neuron(deltaTms, 0, initial_Vm, "input_3")
# neuron_input_4 = Network.Neuron(deltaTms, 0, initial_Vm, "input_4")

neuron_excite_main_0 = Network.Neuron(deltaTms, 0, initial_Vm, "excite_main_0")
neuron_excite_main_1 = Network.Neuron(deltaTms, 0, initial_Vm, "excite_main_1")
neuron_excite_sub_0 = Network.Neuron(deltaTms, 0, initial_Vm, "excite_sub_0")
neuron_excite_sub_1 = Network.Neuron(deltaTms, 0, initial_Vm, "excite_sub_1")

neuron_inhibit_main_0 = Network.Neuron(deltaTms, 0, initial_Vm, "inhibit_main_0")
neuron_inhibit_main_1 = Network.Neuron(deltaTms, 0, initial_Vm, "inhibit_main_1")
neuron_inhibit_sub_0 = Network.Neuron(deltaTms, 0, initial_Vm, "inhibit_sub_0")
neuron_inhibit_sub_1 = Network.Neuron(deltaTms, 0, initial_Vm, "inhibit_sub_1")

neuron_output_0 = Network.Neuron(deltaTms, 0, initial_Vm, "output_0")
neuron_output_1 = Network.Neuron(deltaTms, 0, initial_Vm, "output_1")
# neuron_output_2 = Network.Neuron(deltaTms, 0, initial_Vm, "output_2")

neurons = [neuron_input_0, neuron_input_1, neuron_input_2, #neuron_input_3, neuron_input_4,
            neuron_excite_main_0, neuron_excite_main_1, 
            neuron_excite_sub_0, neuron_excite_sub_1, 
            neuron_inhibit_main_0, neuron_inhibit_main_1, 
            neuron_inhibit_sub_0, neuron_inhibit_sub_1,
            neuron_output_0, neuron_output_1#, neuron_output_2
            ]

neuron_names = ["input_0", "input_1", "input_2", #"input_3", "input_4",
                "excite_main_0", "excite_main_1", 
                "excite_sub_0", "excite_sub_1",
                "inhibit_main_0", "inhibit_main_1",
                "inhibit_sub_0", "inhibit_sub_1",
                "output_0", "output_1"#, "output_2"
                ]



#*********************full layer***************************
# ----------------first input layer------------------------
create_synapse(neuron_input_0, neuron_excite_main_0, "AMPA")
create_synapse(neuron_input_1, neuron_excite_main_0, "AMPA")
create_synapse(neuron_input_2, neuron_excite_main_0, "AMPA")
# create_synapse(neuron_input_3, neuron_excite_main_0, "AMPA")
# create_synapse(neuron_input_4, neuron_excite_main_0, "AMPA")

create_synapse(neuron_input_0, neuron_excite_main_1, "AMPA")
create_synapse(neuron_input_1, neuron_excite_main_1, "AMPA")
create_synapse(neuron_input_2, neuron_excite_main_1, "AMPA")
# create_synapse(neuron_input_3, neuron_excite_main_1, "AMPA")
# create_synapse(neuron_input_4, neuron_excite_main_1, "AMPA")

create_synapse(neuron_input_0, neuron_inhibit_main_0, "GABA")
create_synapse(neuron_input_1, neuron_inhibit_main_0, "GABA")
create_synapse(neuron_input_2, neuron_inhibit_main_0, "GABA")
# create_synapse(neuron_input_3, neuron_inhibit_main_0, "GABA")
# create_synapse(neuron_input_4, neuron_inhibit_main_0, "GABA")

create_synapse(neuron_input_0, neuron_inhibit_main_1, "GABA")
create_synapse(neuron_input_1, neuron_inhibit_main_1, "GABA")
create_synapse(neuron_input_2, neuron_inhibit_main_1, "GABA")
# create_synapse(neuron_input_3, neuron_inhibit_main_1, "GABA")
# create_synapse(neuron_input_4, neuron_inhibit_main_1, "GABA")

# ----------------self recurrent layer----------------
create_synapse(neuron_excite_main_0, neuron_excite_sub_0, "AMPA+NMDA")
create_synapse(neuron_excite_main_0, neuron_excite_sub_1, "AMPA+NMDA")

create_synapse(neuron_excite_main_1, neuron_excite_sub_0, "AMPA+NMDA")
create_synapse(neuron_excite_main_1, neuron_excite_sub_1, "AMPA+NMDA")

create_synapse(neuron_excite_sub_0, neuron_excite_main_0, "AMPA+NMDA")
create_synapse(neuron_excite_sub_0, neuron_excite_main_1, "AMPA+NMDA")

create_synapse(neuron_excite_sub_1, neuron_excite_main_0, "AMPA+NMDA")
create_synapse(neuron_excite_sub_1, neuron_excite_main_1, "AMPA+NMDA")
    

create_synapse(neuron_inhibit_main_0, neuron_inhibit_sub_0, "GABA")
create_synapse(neuron_inhibit_main_0, neuron_inhibit_sub_1, "GABA")

create_synapse(neuron_inhibit_main_1, neuron_inhibit_sub_0, "GABA")
create_synapse(neuron_inhibit_main_1, neuron_inhibit_sub_1, "GABA")

create_synapse(neuron_inhibit_sub_0, neuron_inhibit_main_0, "GABA")
create_synapse(neuron_inhibit_sub_0, neuron_inhibit_main_1, "GABA")

create_synapse(neuron_inhibit_sub_1, neuron_inhibit_main_0, "GABA")
create_synapse(neuron_inhibit_sub_1, neuron_inhibit_main_1, "GABA")

# --------------between excitatory and inhibitory----------------
create_synapse(neuron_excite_main_0, neuron_inhibit_main_0, "AMPA+NMDA")
create_synapse(neuron_excite_main_0, neuron_inhibit_main_1, "AMPA+NMDA")

create_synapse(neuron_excite_main_1, neuron_inhibit_main_0, "AMPA+NMDA")
create_synapse(neuron_excite_main_1, neuron_inhibit_main_1, "AMPA+NMDA")

create_synapse(neuron_inhibit_main_0, neuron_excite_main_0, "GABA")
create_synapse(neuron_inhibit_main_0, neuron_excite_main_1, "GABA")

create_synapse(neuron_inhibit_main_1, neuron_excite_main_0, "GABA")
create_synapse(neuron_inhibit_main_1, neuron_excite_main_1, "GABA")

# ----------------output layer----------------------
create_synapse(neuron_excite_main_0, neuron_output_0, "AMPA")
create_synapse(neuron_excite_main_1, neuron_output_0, "AMPA")

create_synapse(neuron_excite_main_0, neuron_output_1, "AMPA")
create_synapse(neuron_excite_main_1, neuron_output_1, "AMPA")

# create_synapse(neuron_excite_main_0, neuron_output_2, "AMPA")
# create_synapse(neuron_excite_main_1, neuron_output_2, "AMPA")